In [1]:
#encoding=utf-8
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
# import lightgbm as lgb
from sklearn import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import MultinomialNB

import pickle

from all_model_py import CutDebt, IDClassifier, IfKnowDebtor, Installment, WillingToPay, ConfirmLoan

# import sys
# sys.path.append('../CutDebt/')
# sys.path.append('../IDClassifier/')
# sys.path.append('../IfKnowDebtor/')
# sys.path.append('../Installment/')
# sys.path.append('../SetDueDay/')
# sys.path.append('../WillingToPay/')

# from CutDebt_py import CutDebt
# from IDClassifier_py import IDClassifier
# from IfKnowDebtor_py import IfKnowDebtor
# from Installment_py import Installment
# from SetDueDay_py import SetDueDay
# from WillingToPay_py import WillingToPay

In [2]:
model_list = ['CutDebt','IDClassifier','IfKnowDebtor','Installment','ConfirmLoan','WillingToPay']
func_list = [CutDebt,IDClassifier,IfKnowDebtor,Installment,ConfirmLoan,WillingToPay]

ind = 0
for each_model in model_list:
    print(each_model)
    
    # load data
    path = '../../data/{}/'
    data_train = pd.read_csv(path.format(each_model) + 'cleaned_mock_up_data.csv', encoding='utf8')
    print(data_train.label.value_counts())

    
    # get tfidf
    phrase_vectorizer = TfidfVectorizer(ngram_range=(1,3),
                                    strip_accents='unicode', 
                                    max_features=100000, 
                                    analyzer='word',
                                    sublinear_tf=True,
                                    token_pattern=r'\w{1,}')

    print('fitting phrase')
    phrase_vectorizer.fit(data_train.split_text)

    print('transform phrase')
    phrase = phrase_vectorizer.transform(data_train.split_text)

    
    # linear svc
    l_svc = LinearSVC()
    lsvc = CalibratedClassifierCV(l_svc) 
    lsvc.fit(phrase, data_train.label)
    
    
    # logistic
    log_r = LogisticRegression()
    log_r.fit(phrase, data_train.label)
    
    
    # Naive Bayes
    naive_b = MultinomialNB()
    naive_b.fit(phrase, data_train.label)
    
    
    print('finish training')
    
    
    
    
    
#     for threhold
    
    # save model
    save_path = '../../savedModel/{}/'
#     # save tfidf
#     pickle.dump(phrase_vectorizer, open(save_path.format(each_model) + "tfidf.pickle", "wb"))
#     # save linear svc
#     pickle.dump(lsvc, open(save_path.format(each_model) + "LinearSVC.pickle", "wb"))
#     # save logistic
#     pickle.dump(log_r, open(save_path.format(each_model) + "Logistic.pickle", "wb"))
#     # save lightGBM
# #     pickle.dump(lgbm_model, open(save_path.format(each_model) + "Lgbm.pickle", "wb"))
#     # save naive bayes
#     pickle.dump(naive_b, open(save_path.format(each_model) + "nb.pickle", "wb"))

    
    other = pickle.load(open('../../savedModel/others/{}/{}_other.pickle'.format(each_model,each_model),'rb'))
    
    func = func_list[ind]
    result = func(svc=lsvc, logistic=log_r, nb=naive_b, tfidf=phrase_vectorizer, other=other,  jieba_path='../WordCut/userdict.txt')
    pickle.dump(result, open(save_path.format(each_model) + each_model + '.pickle', "wb"))
    ind = ind + 1
    
    
    
    

CutDebt
0    4073
2    2180
1    2088
Name: label, dtype: int64
fitting phrase
transform phrase


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


finish training


Loading model cost 0.622 seconds.
Prefix dict has been built succesfully.


IDClassifier
0    1427
2    1293
1    1272
Name: label, dtype: int64
fitting phrase
transform phrase
finish training
IfKnowDebtor
0    1461
2    1295
1    1156
Name: label, dtype: int64
fitting phrase
transform phrase
finish training
Installment
0    4073
2    2180
1    2088
Name: label, dtype: int64
fitting phrase
transform phrase
finish training
ConfirmLoan
2    283
1    212
0    201
Name: label, dtype: int64
fitting phrase
transform phrase
finish training
WillingToPay
0    2089
2    2053
1    2042
3    1273
Name: label, dtype: int64
fitting phrase
transform phrase
finish training


In [3]:
# import jieba
# sentence = '好'
# sentence = jieba.cut(sentence)
# sentence = ' '.join(sentence)
# sentence = phrase_vectorizer.transform([sentence])
# naive_b.predict_proba(sentence)

In [4]:
idc = pickle.load(open("../../savedModel/IDClassifier/IDClassifier.pickle", 'rb'))
cutd = pickle.load(open("../../savedModel/CutDebt/CutDebt.pickle", 'rb'))
ifk = pickle.load(open("../../savedModel/IfKnowDebtor/IfKnowDebtor.pickle", 'rb'))
will = pickle.load(open("../../savedModel/WillingToPay/WillingToPay.pickle", 'rb'))
inst = pickle.load(open("../../savedModel/Installment/Installment.pickle", 'rb'))
conf = pickle.load(open("../../savedModel/ConfirmLoan/ConfirmLoan.pickle", 'rb'))

In [5]:
will.classify('我还不了',lower_bounder=36,upper_bounder=24*3)

ML output is : 1


{'av_pred': array([ 0.05529326,  0.81661152,  0.0963272 ,  0.03176801]),
 'label': 1,
 'pred_prob': array([[ 0.0317503 ,  0.83436165,  0.09123825,  0.04264979],
        [ 0.08291824,  0.78695503,  0.09520352,  0.03492321],
        [ 0.05121125,  0.82851789,  0.10253982,  0.01773104]]),
 'time_extract': []}

In [9]:
will.classify('我明天不能还后天可以',lower_bounder=36,upper_bounder=24*3)

{'av_pred': 1.0,
 'label': 10,
 'pred_prob': 1.0,
 'time_extract': [{'gapH': 20.142228813888888,
   'gapS': 72512.02373,
   'pattern': '明天',
   'time': datetime.datetime(2018, 7, 3, 16, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>)},
  {'gapH': 44.142228813888885,
   'gapS': 158912.02373,
   'pattern': '后天',
   'time': datetime.datetime(2018, 7, 4, 16, 0, tzinfo=<DstTzInfo 'America/New_York' EDT-1 day, 20:00:00 DST>)}]}

In [1]:
will.re_time.remove_time('我明天还不了')

NameError: name 'will' is not defined